## Exercises: Heavy Lifting

Add `fmap`, parentheses, and function composition to the expression as needed for the expression to typecheck and produce the expected result. It may not always need to go in the same place, so don’t get complacent.

In [1]:
-- 1.
a = fmap (+1) (read "[1]" :: [Int])
a -- [2]

-- 2.
b = (fmap . fmap) (++ "lol") (Just ["Hi,", "Hello"])
b -- Just ["Hi,lol","Hellolol"]

-- 3.
c = fmap (*2) (\x -> x - 2)
c 1 -- -2

-- 4.
d =
    fmap
    ((return '1' ++) . show)
    (\x -> [x, 1..3])
d 0 -- "1[0,1,2,3]"

-- 5.
e :: IO Integer
e = let ioi = readIO "1" :: IO Integer
        changed = fmap (read . ("123"++) . show) ioi
    in fmap (*3) changed
e -- 3693

[2]

Just ["Hi,lol","Hellolol"]

-2

"1[0,1,2,3]"

3693

## Exercises: Instances of Func

Implement `Functor` instances for the following datatypes. Use the `QuickCheck` properties we showed you to validate them.

In [59]:
import Test.QuickCheck
import Test.QuickCheck.Function

--1.

newtype Identity a =
    Identity a
    deriving (Eq, Show)

instance Functor Identity where
    fmap f (Identity a) = Identity (f a)

instance Arbitrary a => Arbitrary (Identity a) where
    arbitrary =
        do  x <- arbitrary
            return $ Identity x

--2. data Pair a = Pair a a

data Pair a =
    Pair a a
    deriving (Eq, Show)

instance Functor Pair where
    fmap f (Pair a b) = Pair (f a) (f b)

instance Arbitrary a => Arbitrary (Pair a) where
    arbitrary =
        do  x <- arbitrary
            y <- arbitrary
            return $ Pair x y

--3. data Two a b = Two a b

data Two a b =
    Two a b
    deriving (Eq, Show)

instance Functor (Two a) where
    fmap f (Two a b) = Two a (f b)

instance (Arbitrary a, Arbitrary b) => Arbitrary (Two a b) where
    arbitrary =
        do  x <- arbitrary
            y <- arbitrary
            return $ Two x y

--4. data Three a b c = Three a b c
--5. data Three' a b = Three' a b b
--6. data Four a b c d = Four a b c d
--7. data Four' a b = Four' a a a b
--8. data Trivial = Trivial

--

functorIdentity :: (Functor f, Eq (f a)) => f a -> Bool
functorIdentity f =
    fmap id f == f

functorCompose :: (Eq (f c), Functor f) => f a -> Fun a b -> Fun b c -> Bool
functorCompose x (Fun _ f) (Fun _ g) =
    (fmap (g . f) x) == (fmap g . fmap f $ x)

--

quickCheck (functorIdentity :: Identity String -> Bool)
quickCheck (functorCompose :: Identity String -> Fun String Int -> Fun Int String -> Bool)
quickCheck (functorIdentity :: Pair String -> Bool)
quickCheck (functorCompose :: Pair String -> Fun String Int -> Fun Int String -> Bool)
quickCheck (functorIdentity :: Two Char String -> Bool)
quickCheck (functorCompose :: Two Char String -> Fun String Int -> Fun Int String -> Bool)

+++ OK, passed 100 tests.

+++ OK, passed 100 tests.

+++ OK, passed 100 tests.

+++ OK, passed 100 tests.

+++ OK, passed 100 tests.

+++ OK, passed 100 tests.

## Exercise: Possibly & Sum

1. Write a `Functor` instance for a datatype identical to `Maybe`. We’ll use our own datatype because `Maybe` already has a `Functor` instance and we cannot make a duplicate one.

2. Write a `Functor` instance for a datatype identical to `Either`. We’ll use our own datatype because `Either` has a `Functor` instance.

3. Why is a `Functor` instance that applies the function only to First, `Either`’s `Left`, impossible? We covered this earlier.

In [5]:
data Possibly a =
    LolNope
    | Yeppers a
    deriving (Eq, Show)

instance Functor Possibly where
    fmap _ LolNope     = LolNope
    fmap f (Yeppers a) = Yeppers (f a)
    
--

fmap (+1) LolNope
fmap (+1) (Yeppers 1)

LolNope

Yeppers 2

In [8]:
data Sum a b =
    First a
    | Second b
    deriving (Eq, Show)

instance Functor (Sum a) where
    fmap _ (First a) = First a
    fmap f(Second a) = Second (f a)

--

fmap (+1) (First "blah")
fmap (+1) (Second 1)

First "blah"

Second 2

## Chapter Exercises

Determine if a valid `Functor` can be written for the datatype provided.

In [ ]:
-- 1. No, kind *

data Bool =
    False
    | True
    
-- 2. Yes, kind * -> *

data BoolAndSomethingElse a =
    False' a
    | True' a

-- 3. Yes, kind * -> *

data BoolAndMaybeSomethingElse a =
    Falsish
    | Truish a

-- 4. Yes, kind * -> *

newtype Mu f =
    InF { outF :: f (Mu f) }

-- 5. No, kind is *

import GHC.Arr

data D =
    D (Array Word Word) Int Int

Rearrange the arguments to the type constructor of the datatype so the `Functor` instance works.

In [30]:
-- 1.

data Sum b a =
    First a
    | Second b
    deriving (Eq, Show)

instance Functor (Sum e) where
    fmap f (First a)  = First (f a)
    fmap f (Second b) = Second b
    
--

fmap (+1) (First 1)
fmap (+1) (Second 1)

-- 2.

data Company a c b =
    DeepBlue a c
    | Something b
    deriving (Eq, Show)

instance Functor (Company e e') where
    fmap f (Something b)  = Something (f b)
    fmap _ (DeepBlue a c) = DeepBlue a c
    
--
    
fmap (+1) (DeepBlue 1 2)
fmap (+1) (Something 1)

-- 3.

data More b a =
    L a b a
    | R b a b
    deriving (Eq, Show)

instance Functor (More x) where
    fmap f (L a b a') =
        L (f a) b (f a')
    fmap f (R b a b') =
        R b (f a) b'

--

fmap (+1) (L 1 2 3)
fmap (+1) (R 1 2 3)

First 2

Second 1

DeepBlue 1 2

Something 2

L 2 2 4

R 1 3 3

In [42]:
{-# LANGUAGE FlexibleInstances #-}

-- 1.

data Quant a b =
    Finance
    | Desk a
    | Bloor b
    deriving (Eq, Show)

instance Functor (Quant a) where
    fmap f Finance   = Finance
    fmap f (Desk a)  = Desk a
    fmap f (Bloor b) = Bloor (f b)
    
-- 2.

data K a b =
    K a
    deriving (Eq, Show)

instance Functor (K a) where
    fmap f (K a) = K a
    
-- 3.

newtype Flip f a b =
    Flip (f b a)
    deriving (Eq, Show)

newtype K' a b =
    K' a
    deriving (Eq, Show)

instance Functor (Flip K' a) where
    fmap f (Flip (K' a)) = Flip $ K' (f a)
    
-- 4.

data EvilGoateeConst a b =
    GoatyConst b
    deriving (Eq, Show)

instance Functor (EvilGoateeConst a) where
    fmap f (GoatyConst b) = GoatyConst (f b)
    
-- 5.

data LiftItOut f a =
    LiftItOut (f a)
    deriving (Eq, Show)

instance Functor f => Functor (LiftItOut f) where
    fmap f (LiftItOut fa) = LiftItOut (fmap f fa)
    
-- 6.

data Parappa f g a =
    DaWrappa (f a) (g a)
    deriving (Eq, Show)

instance (Functor f, Functor g) => Functor (Parappa f g) where
    fmap f (DaWrappa fa ga) =
        DaWrappa (fmap f fa) (fmap f ga)

-- 7.

data IgnoreOne f g a b =
    IgnoreSomething (f a) (g b)
    deriving (Eq, Show)

instance Functor g => Functor (IgnoreOne f g a) where
    fmap f (IgnoreSomething a ga) =
        IgnoreSomething a (fmap f ga)
        
-- 8.

data Notorious g o a t =
    Notorious (g o) (g a) (g t)
    deriving (Eq, Show)

instance Functor g => Functor (Notorious g o a) where
    fmap f (Notorious o a gt) =
        Notorious o a (fmap f gt)
        
-- 9.

data List a =
    Nil
    | Cons a (List a)
    deriving (Eq, Show)

instance Functor List where
    fmap f Nil        = Nil
    fmap f (Cons a xs) =
        Cons (f a) (fmap f xs)
        
-- 10.

data GoatLord a =
    NoGoat
    | OneGoat a
    | MoreGoats (GoatLord a) (GoatLord a) (GoatLord a)
    deriving (Eq, Show)

instance Functor GoatLord where
    fmap f NoGoat            = NoGoat
    fmap f (OneGoat a)       = OneGoat (f a)
    fmap f (MoreGoats x y z) =
        MoreGoats (fmap f x) (fmap f y) (fmap f z)